# Object-Level CND Annotations

This notebook demonstrates object-level CND annotations, allowing annotations to be applied to specific object instances rather than just their classes. This solves the core issue where users want to annotate individual objects (like sets) without modifying the class definition.


In [19]:
import sys
from pathlib import Path

# Add the parent directory to the Python path
sys.path.append(str(Path().resolve().parent))

from spytial import (
    diagram, 
    orientation, cyclic, group, atomColor  # Now work on both classes AND objects!
)

## Demo 1: Set Grouping 

This demonstrates the core issue request: annotating specific objects (like sets) without modifying their classes.


In [20]:
# Create different sets - each can be annotated differently
fruits = {"apple", "banana", "cherry", "date"}
numbers = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
colors = {"red", "green", "blue"}


fruits = group(field='contains', selector='self', groupOn=0, addToGroup=1)(fruits)


numbers = group(field='contains', selector='self', groupOn=0, addToGroup=1)(numbers)


# Leave colors set without special annotations




In [21]:
# Visualize the sets with their object-specific annotations

# Create a container to show them all
set_data = {
    "fruits": fruits,
    "numbers": numbers, 
    "colors": colors,
}
diagram(set_data)

## Demo 2: Class + Object Annotations Working Together

This shows how class-level decorators and object-level decorators can combine seamlessly using the same decorator syntax.

In [22]:
# Class-level decorators (traditional usage)
@orientation(selector='left', directions=['left, below'])
@orientation(selector='right', directions=['left, below'])
class TreeNode:
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

# Create a tree structure
root = TreeNode("root")
root.left = TreeNode("left-child")
root.right = TreeNode("right-child")
root.left.left = TreeNode("left-left")
root.left.right = TreeNode("left-right")

# Object-level decorators (NEW ergonomic usage - same syntax!)
root = atomColor(selector='self', value='red')(root)

root.left = orientation(selector='self', directions=['below'])(
    group(field='subtree', groupOn=0, addToGroup=1)(root.left)
)

print("✅ Class annotations apply to ALL TreeNode instances")
print("✅ Object annotations apply only to specific instances")
print("✅ Both use the SAME decorator syntax!")
print("\n🎯 Much more intuitive - Python developers expect decorators to 'just work'")

✅ Class annotations apply to ALL TreeNode instances
✅ Object annotations apply only to specific instances
✅ Both use the SAME decorator syntax!

🎯 Much more intuitive - Python developers expect decorators to 'just work'


In [23]:
# Visualize the tree with mixed annotations
diagram(root)

## Demo 3: Built-in Types Annotations

Demonstrates annotations on various built-in types that can't normally store attributes.

## Demo 4: Object-Level Custom Providers

You can also set custom data providers for specific objects, controlling how they are serialized to atoms and relations.

In [24]:
from spytial import DataInstanceProvider, object_provider, set_object_provider

# Define a custom provider for sets
class CustomSetProvider(DataInstanceProvider):
    def can_handle(self, obj):
        return isinstance(obj, set)
    
    def provide_atoms_and_relations(self, obj, walker_func):
        obj_id = walker_func._get_id(obj)
        atom = {
            "id": obj_id,
            "type": "custom_set",
            "label": f"MySet[{len(obj)}]"
        }
        
        relations = []
        for i, item in enumerate(obj):
            item_id = walker_func(item)
            relations.append((f"element_{i}", obj_id, item_id))
        
        return atom, relations

# Method 1: Decorator syntax
my_set = {1, 2, 3, 4, 5}
my_set = object_provider(CustomSetProvider())(my_set)

# Method 2: Function call
my_other_set = {6, 7, 8}
set_object_provider(my_other_set, CustomSetProvider())

# Method 3: Combine with annotations
custom_annotated_set = object_provider(CustomSetProvider())(
    group(field='elements', groupOn=0, addToGroup=1)({10, 11, 12})
)

print("✅ Custom providers set for different sets")
print("- my_set: uses CustomSetProvider")
print("- my_other_set: also uses CustomSetProvider")
print("- custom_annotated_set: CustomSetProvider + group annotation")
print("\n🎯 Object-specific providers override default serialization!")

✅ Custom providers set for different sets
- my_set: uses CustomSetProvider
- my_other_set: also uses CustomSetProvider
- custom_annotated_set: CustomSetProvider + group annotation

🎯 Object-specific providers override default serialization!


In [25]:
# Create a comparison structure
provider_demo = {
    "normal_set": {"a", "b", "c"},  # Uses default provider
    "custom_set1": my_set,  # Uses custom provider
    "custom_set2": my_other_set,  # Uses custom provider
    "custom_annotated": custom_annotated_set  # Custom provider + annotations
}

# Visualize to see the difference
diagram(provider_demo)

### Custom Provider Benefits

- **Override default serialization**: Replace how specific objects are converted to atoms/relations
- **Object-specific behavior**: Different instances of the same class can use different providers
- **Combine with annotations**: Custom providers work alongside spatial annotations
- **Maintain type flexibility**: Keep the benefits of Python's dynamic typing while customizing visualization